In [ ]:
!pip install -q ipyvolume

     |████████████████████████████████| 2.9 MB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 18.9 MB/s 
     |████████████████████████████████| 260 kB 40.5 MB/s 
     |████████████████████████████████| 1.6 MB 34.6 MB/s 
     |████████████████████████████████| 271 kB 53.0 MB/s 


In [ ]:
#@title Import Packages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=TypeError)
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Layout
import ipyvolume as ipv
from IPython import display
from time import sleep
from termcolor import cprint
import pythreejs
from os.path import exists
from google.colab import files
from google.colab import output
output.enable_custom_widget_manager()
%config InlineBackend.figure_format = 'svg'

# Standard PIB Solutions

$$\psi_{n_x,n_y,n_z}=\sqrt{\frac{8}{l_xl_yl_z}}\sin\bigg(\frac{n_x\pi x}{l_x}\bigg)\sin\bigg(\frac{n_y\pi y}{l_y}\bigg)\sin\bigg(\frac{n_z\pi z}{l_z}\bigg)\\
E_{n_x,n_y,n_z}= \frac{\pi^2}{2}\bigg[\bigg(\frac{n_x}{l_x}\bigg)^2 + \bigg(\frac{n_y}{l_y}\bigg)^2 + \bigg(\frac{n_z}{l_z}\bigg)^2\bigg]\\
$$


In [ ]:
#@title 
def psi_reg(x, y, z, q_nx=1, q_ny=1, q_nz=1,lx=1,ly=1,lz=1):
    wvfn = np.sqrt(8/(lx*ly*lz)) * \
    np.sin((q_nx*np.pi*x)/lx) * \
    np.sin((q_ny*np.pi*y)/ly) * \
    np.sin((q_nz*np.pi*z)/lz)
    return wvfn

def psi_ener(q_nx=1, q_ny=1, q_nz=1, lx=1, ly=1, lz=1):
    e_level = (np.pi**2/2)*((q_nx/lx)**2 + (q_ny/ly)**2 + (q_nz/lz)**2)
    print(e_level)

num_elect_slider = widgets.Dropdown(options=np.arange(2,27,2),value=10,description='electrons:',disabled=False)
lx_slider = widgets.IntSlider(value=5,min=1,max=20,step=1,description='lx',disabled=False,readout_format='d',continuous_update=False)
ly_slider = widgets.IntSlider(value=5,min=1,max=20,step=1,description='ly',disabled=False,readout_format='d',continuous_update=False)
lz_slider = widgets.IntSlider(value=5,min=1,max=20,step=1,description='lz',disabled=False,readout_format='d',continuous_update=False)
nx_slider = widgets.IntSlider(value=1,min=1,max=10,step=1,description='nx',disabled=False,readout_format='d',continuous_update=False)
ny_slider = widgets.IntSlider(value=1,min=1,max=10,step=1,description='ny',disabled=False,readout_format='d',continuous_update=False)
nz_slider = widgets.IntSlider(value=1,min=1,max=10,step=1,description='nz',disabled=False,readout_format='d',continuous_update=False)

filename_text_mpl = widgets.Text(description='Filename (.png): ',value='PIB_ener',style={'description_width': 'initial'})
filename_text_ipv = widgets.Text(description='Filename (.png): ',value='PIB_iso',style={'description_width': 'initial'})
save_button_mpl = widgets.Button(description='Save Image')
save_button_ipv = widgets.Button(description='Save Image')

In [ ]:
#@title Energy Level Plot (execute cell first)
def psi_ener(qnx, qny, qnz, lx, ly, lz):
    e_level = (4*np.pi**2/8)*((qnx/lx)**2 + (qny/ly)**2 + (qnz/lz)**2)
    return e_level

def PIB_plotter(lx,ly,lz,num_elect):
    ener_list = []

    for i in range(1,20):
        for j in range(1,20):
            for k in range(1,20):
                ener_list.append(((i,j,k),psi_ener(qnx=i,qny=j,qnz=k, lx=lx, ly=ly, lz=lz)))

    ener_list.sort(key=lambda x: abs(x[1]))
    ener_list = np.asarray(ener_list,dtype=object)
    ener_list[:,1] = ener_list[:,1].astype(dtype=float)

    degen_list = np.unique(ener_list[:,1],return_counts=True)

    degen_log = np.array([],dtype=int)

    for i in degen_list[1]:
        degen_log = np.append(degen_log, i*np.ones(i,dtype=int))

    degen_log = degen_log.reshape(len(degen_log),1)

    ener_list = np.hstack((ener_list,degen_log))

    #################find the unoccupied levels##############
    occ_levels = int((num_elect/ 2))
    occ_states = ener_list[0:occ_levels]
    occ_degen_accounted = np.where(occ_states[-1,1] == occ_states[:,1])[0].size
    occ_state_miss = occ_states[-1,2] - occ_degen_accounted
    if occ_state_miss > 0:
        occ_levels = occ_levels + occ_state_miss
        occ_states = ener_list[0:occ_levels]
    ##################find the occupied levesl############
    unocc_levels = 2
    unocc_states = ener_list[occ_levels:occ_levels+unocc_levels]
    unocc_degen_accounted = np.where(unocc_states[-1,1] == unocc_states[:,1])[0].size
    unocc_state_miss = unocc_states[-1,2] - unocc_degen_accounted
    if unocc_state_miss > 0:
        unocc_levels = unocc_levels + unocc_state_miss
        unocc_states = ener_list[occ_levels:occ_levels + unocc_levels]
    occ = []
    unocc = []
    energy_PIB = []
    for state in occ_states:
        occ.append(state[0])
        energy_PIB.append(round(state[1],8))
    for state in unocc_states:
        unocc.append(state[0])
        energy_PIB.append(round(state[1],8))
    PIBlevels = occ + unocc

    yPIB = np.array(energy_PIB)
    xPIB = np.ones(yPIB.shape[0])
    for i in range(0,len(yPIB)):
        #print(yPIB[i])
        if yPIB[i] in yPIB[:i]:
            #print('Yes')
            #print(yPIB[:i])
            count = list(yPIB[:i]).count(yPIB[i])
            #print(count)
            xPIB[i] += count*0.3

    Gap_PIB = round(energy_PIB[len(occ)] - energy_PIB[len(occ)-1], 3)
    Gap_PIB_ev = round(Gap_PIB*27.2114, 3)    
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    plt.cla()
    display.clear_output(wait=True)
    plt.ylabel("Energy (Ha)",labelpad=7)
    plt.scatter(xPIB[len(occ):],yPIB[len(occ):],marker=0,s=1200,linewidths=6, color='#F97306', label='virtual')
    plt.scatter(xPIB[:len(occ)],yPIB[:len(occ)],marker=0,s=1200,linewidths=6, color='green', label='occupied')
    plt.rcParams["legend.markerscale"] = 0.35
    plt.legend(loc='upper right')
    plt.xticks([])
    plt.xlim([-0.1,3])
    plt.ylim([min(yPIB)-0.1,max(yPIB)+0.1])
    annotations = [str(x) for x in PIBlevels]
    for i, label in enumerate(annotations):
          if list(yPIB).count(yPIB[i])==1:
              plt.annotate(label, (xPIB[i] + 0.005, yPIB[i]),size=8)
              plt.text(xPIB[i]-0.46, yPIB[i], "{:.3f}".format(yPIB[i]),size=8)
          else:
              plt.annotate(label, (xPIB[i] - 0.25, yPIB[i] - 0.015),size=8)
              if yPIB[i] not in yPIB[:i]:
                  plt.text(xPIB[i]-0.46, yPIB[i], "{:.3f}".format(yPIB[i]),size=8)
                  #plt.text(xt[i]-0.2, yt[i] + 0.015, "{:.3f}".format(yt[i]),size=8)
    plt.text(xPIB[0]+0.4, max(yPIB)+0.13, 'PIB', size=12)
    plt.text(xPIB[0]-0.04, max(yPIB)+0.10,f'HOMO-LUMO Gap: {Gap_PIB_ev} eV', size=10.5)
    ax.spines['left'].set_position(('axes', .16))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    plt.tight_layout()

def on_click_save(b):
  PIB_plotter(lx_slider.value,ly_slider.value,lz_slider.value,num_elect_slider.value)
  plt.tight_layout()
  plt.savefig(f'{filename_text_mpl.value}.png',dpi=800)
  plt.close()

save_button_mpl.on_click(on_click_save)


widgets.interact(PIB_plotter, lx=lx_slider,ly=ly_slider,lz=lz_slider,num_elect=num_elect_slider)
widgets.HBox([filename_text_mpl,save_button_mpl])


interactive(children=(IntSlider(value=5, continuous_update=False, description='lx', max=20, min=1), IntSlider(…

In [ ]:
#@title Isosurface Plotting
lx = lx_slider.value
ly = ly_slider.value
lz = lz_slider.value
def isoplotter(nx_val,ny_val,nz_val,psi_square=True):
  

  nx_p, ny_p, nz_p = 7 * lx, 7 * ly, 7 * lz
  xp = np.linspace(0, lx, nx_p)
  yp = np.linspace(0, ly, ny_p)
  zp = np.linspace(0, lz, nz_p)

#construct 3d grid of points
  X, Y, Z = np.meshgrid(xp, yp, zp, indexing='ij')

  psi = psi_reg(X,Y,Z,nx_val,ny_val,nz_val,lx,ly,lz)

  norm_psi = psi_reg(X,Y,Z,nx_val,ny_val,nz_val,lx,ly,lz)**2

  lengths = f'lengths: lx = {lx}, ly = {ly}, lz = {lz} (Bohr)'
  cprint('     ' + f'3D PIB Wavefunction Isosurface',attrs=['bold']) 
  print(len(lengths)*'=')
  cprint(f'lengths: lx = \033[1m\033[94m{lx}\033[0m, ly = \033[1m\033[94m{ly}\033[0m, lz = \033[1m\033[94m{lz}\033[0m (Bohr)')
  print(len(lengths)*'-')
  cprint(f'state:   nx = \033[1m\033[36m{nx_val}\033[0m, ny = \033[1m\033[36m{ny_val}\033[0m, nz = \033[1m\033[36m{nz_val}\033[0m')
  print(len(lengths)*'-')
  ipv.clear()
  fig = ipv.figure(title='PIB',width=800, height=800)
  fig.camera.type = 'OrthographicCamera'


  if psi_square:
    norm_sur = ipv.pylab.plot_isosurface(norm_psi,color='red',level=norm_psi.mean(),controls=False)
  else:
      pos_values = np.ma.array(psi, mask = psi < 0.0)
      if nx_val == ny_val == nz_val == 1:
        pos_sur = ipv.pylab.plot_isosurface(psi,color='red',level=np.sqrt(norm_psi.mean()),controls=True)
      else:
        pos_sur = ipv.pylab.plot_isosurface(psi,color='red',level=np.sqrt(norm_psi.mean()),controls=True)
        neg_sur = ipv.pylab.plot_isosurface(psi,color='blue',level=-np.sqrt(norm_psi.mean()),controls=True)


  ipv.style.box_off()
  ipv.squarelim()
  ipv.view(0,-75)
  ipv.xyzlabel(r'lx','ly','lz')

  ipv.show()
  #print(' \033[4misovalue slider (e/Bohr**3)')

def plot_saver(b):
  ipv.savefig(f'{filename_text_ipv.value}.png',width=1200,height=1200)

save_button_ipv.on_click(plot_saver)

widgets.interact(isoplotter,nx_val=nx_slider,ny_val=ny_slider,nz_val=nz_slider)

widgets.HBox([filename_text_ipv,save_button_ipv])

interactive(children=(IntSlider(value=1, continuous_update=False, description='nx', max=10, min=1), IntSlider(…

Output()